In [1]:
import pandas as pd
pd.set_option("max_columns", None)

In [3]:
from lxml import etree

In [4]:
tammunition = etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TAmmunition.xml")

In [8]:
demo_element = tammunition.findall("TAmmunition")[0]

In [12]:
demo_element.items()

[('id', '16533')]

In [20]:
demo_sub_element = demo_element.iterchildren().__next__()

In [24]:
demo_sub_element.tag

'DescriptorId'

In [23]:
demo_sub_element.text

'00000000-0000-0000-0600-000031080000'

In [25]:
element_dict = dict()
for element in tammunition.findall("TAmmunition")[0]:
    tag, text = element.tag, element.text
    element_dict[tag] = text

In [26]:
element_dict

{'AffecteParNombre': 'null',
 'AffichageMenu': 'null',
 'AffichageMunitionParSalve': '1',
 'AngleDispersion': '0.1',
 'Arme': '4',
 'Caliber': 'null',
 'CorrectedShotDispersionMultiplier': 'null',
 'DescriptorId': '00000000-0000-0000-0600-000031080000',
 'DispersionAtMaxRange': 'null',
 'DispersionAtMinRange': 'null',
 'EfficaciteSelonPortee': 'null',
 'FX_frottement': '0.001',
 'FX_tir_sans_physic': 'True',
 'FX_tir_tendu': 'True',
 'FX_vitesse_de_depart': '10000',
 'FireDescriptor': '82 : 27774 (True) - TUniteDescriptor',
 'FireTriggeringProbability': '0.15',
 'Guidance': 'null',
 'HitRollRule': '113 : 27773 (False) - TModernWarfareHitRollRule',
 'IgnoreInflammabilityConditions': 'null',
 'InterdireTirReflexe': 'null',
 'InterfaceWeaponTexture': 'null',
 'IsFireAndForget': 'null',
 'IsSubAmmunition': 'null',
 'Level': '5',
 'MissileDescriptor': 'null',
 'MissileTimeBetweenCorrections': 'null',
 'Name': '9CF6380000000000',
 'NbTirParSalves': '1',
 'NbrProjectilesSimultanes': '1',
 'Ne

In [28]:
# pd.Series(element_dict)

In [58]:
tunits = etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TUniteAuSolDescriptor.xml")
weapon_managers = etree.parse("../raws/510049986/NDF_Win/pc/ndf/patchable/gfx/everything/TWeaponManagerModuleDescriptor.xml")

In [74]:
weapon_managers.findall("TWeaponManagerModuleDescriptor[@id='27760']")

[]

In [72]:
def serialize_unit(tunit_xml, weapon_manager_xml):
    id = tunit_xml.items()[0][1]
    unit = pd.Series()
    unit['ID'] = id
    for tunit_element in tunit_xml:
        tag = tunit_element.tag
        # The complex case.
        if tag == "Modules":
            # Find the WeaponManager module ID.
            modules = [collectionelement.text for collectionelement in tunit_element]
            weapon_manager_text = list(filter(lambda module: "WeaponManager" in module, modules))[0]
            weapon_manager_id = weapon_manager_text.split(":")[-1].split(" ")[1]
            # Recursively pass it to serialize_weapon_manager.
            serialize_weapon_manager(unit, weapon_manager_id, weapon_manager_xml)
            pass
        # All other cases are simple.
        elif tag == "ProductionPrice":
            production_prices = [collectionelement.text for collectionelement in tunit_element]
            # e.g. production_prices = [90, 15, 15, 15, 15].
            # This structure seems to allow variation of price based on veterancy, but this feature is not used in-game.
            actual_price = production_prices[0]
            unit['ProductionPrice'] = actual_price
        elif tag == "MaxDeployableAmount":
            deployable_amounts = [collectionelement.text for collectionelement in tunit_element]
            for i, vet_availability in enumerate(['RookieDeployableAmount', 'TrainedDeployableAmount', 
                                               'HardenedDeployableAmount', 'VeteranDeployableAmount',
                                               'EliteDeployableAmount']):
                unit[vet_availability] = deployable_amounts[i]
        # Ignore junk group tags
        elif tag in ["ShowInMenu", "UnitTypeTokens"]:
            pass
        else:
            val = tunit_element.text
            unit[tag] = val
    return unit

def serialize_weapon_manager(unit, weapon_manager_id, weapon_manager_xml):
    print(weapon_manager_id)
    weapon_manager = weapon_manager_xml.findall("TWeaponManagerModuleDescriptor[@id='{0}']".format(weapon_manager_id))[0]
    print(weapon_manager)
#     weapon_manager = list(filter(lambda manager: manager.items()[0][1] == weapon_manager_id, weapon_manager_xml))[0]
#     print(weapon_manager)

In [73]:
serialize_unit(tunits.findall("TUniteAuSolDescriptor")[0], weapon_managers)

16519


IndexError: list index out of range

In [31]:
for descriptor in tunits.findall("TUniteAuSolDescriptor"):
    pass